# RL basics

Термины и понятия:

- агент/среда
- наблюдение $o$ / состояние $s$
- действие $a$, стратегия $\pi: \pi(s) \rightarrow a$ функция перехода $T: T(s, a) \rightarrow s'$
- вознаграждение $r$, ф-я вознаграждений $R: R(s, a) \rightarrow r$
- цикл взаимодействия, траектория $\tau: (s_0, a_0, r_0, s_1, a_1, r_1, ..., s_T, a_T, r_T)$, эпизод
- отдача $G$, подсчет отдачи, средняя[/ожидаемая] отдача $\mathbb{E}[G]$

In [1]:
try:
    import google.colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    pass

if COLAB:
    !pip -q install "gymnasium[classic-control, atari, accept-rom-license]"
    !pip -q install piglet
    !pip -q install imageio_ffmpeg
    !pip -q install moviepy==1.0.3

In [2]:
import glob
import io
import base64
import gymnasium as gym
import numpy as np
from IPython import display as ipythondisplay
from IPython.display import HTML
import matplotlib.pyplot as plt
%matplotlib inline

## Agent, environment

<img src=https://gymnasium.farama.org/_images/lunar_lander.gif caption="lunar lander" width="150" height="50"><img src=https://gymnasium.farama.org/_images/mountain_car.gif caption="mountain car" width="150" height="50">
<img src=https://gymnasium.farama.org/_images/cliff_walking.gif caption="cliff walking" width="300" height="50">
<img src=https://ale.farama.org/_images/montezuma_revenge.gif caption="montezuma revenge" width="150" height="100">
<img src=https://github.com/danijar/crafter/raw/main/media/video.gif caption="crafter" width="150" height="100">
<img src=https://camo.githubusercontent.com/6df2ca438d8fe8aa7a132b859315147818c54af608f8609320c3c20e938acf48/68747470733a2f2f6d656469612e67697068792e636f6d2f6d656469612f344e78376759694d394e44724d724d616f372f67697068792e676966 caption="malmo minecraft" width="150" height="100">
<img src=https://images.ctfassets.net/kftzwdyauwt9/e0c0947f-1a44-4528-4a41450a9f0a/2d0e85871d58d02dbe01b2469d693d4a/table-03.gif caption="roboschool" width="150" height="100">
<img src=https://raw.githubusercontent.com/Tviskaron/mipt/master/2019/RL/02/mdp.png caption="Марковский процесс принятия решений" width="150" height="100">
<img src=https://minigrid.farama.org/_images/DoorKeyEnv.gif caption="minigrid" width="120" height="120">

## Observation, state

TODO:
- добавить примеры наблюдений/состояний (числа, векторы, картинки)
- интуитивное объяснение различия, положить пока, что наблюдение = состояние
- пространство состояний


В каждый момент времени среда имеет некоторое внутреннее состояние. Здесь слово "состояние" я употребил скорее в интуитивном понимании, чтобы обозначить, что среда изменчива (иначе какой смысл с ней взаимодействовать, если ничего не меняется). В обучении с подкреплением под термином состояние $s$ (или $s_t$, где $t$ — текущее время) подразумевают либо абстрактно информацию о "состоянии" среды, либо ее явное представление в виде данных, достаточные для полного описания "состояния". *NB: Здесь можно провести аналогию с компьютерными играми — файл сохранения игры как раз содержит информацию о "состоянии" мира игры, чтобы в будущем можно было продолжить с текущей точки, так что данные этого файла в целом можно с некоторой натяжкой считать состоянием (с натяжкой, потому что редко когда в сложных играх файлы сохранения содержат прямо вот всю информацию, так что после перезагрузки вы получите не совсем точную копию). При этом обычно подразумевается, что состояние не содержит в себе ничего лишнего, то есть это **минимальный** набор информации.*

Наблюдением $o$ называют то, что агент "видит" о текущем состоянии среды. Это не обязательно зрение, а вообще вся доступная ему информация (условно, со всех его органов чувств).

В общем случае наблюдение: кортеж/словарь многомерных векторов чисел.

In [3]:
print(gym.make("CartPole-v0").reset()[0].shape)
print(gym.make("MountainCar-v0").reset()[0].shape)

(4,)
(2,)


/home/vagiz/Desktop/desktop_vagiz/HSE/Programming/RL/.venv/lib/python3.12/site-packages/gymnasium/envs/registration.py:512: DeprecationWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(


## Action, policy, transition function

Рассмотрим следующие MDP:

- A: <img src=https://i.ibb.co/mrCMVZLQ/mdp-a.png caption="A" width="400" height="100">
- B: <img src=https://i.ibb.co/GQ2tVtjC/mdp-b.png caption="B" width="400" height="100">

Links to all:
[A](https://i.ibb.co/mrCMVZLQ/mdp-a.png)
[B](https://i.ibb.co/GQ2tVtjC/mdp-b.png)
[C](https://i.ibb.co/Jj9LYHjP/mdp-c.png)
[D](https://i.ibb.co/Y47Mr83b/mdp-d.png)
[E](https://i.ibb.co/Kjt1Xhmf/mdp-e.png)

Давайте явно запишем пространства состояний $S$ и действий $A$, а также функцию перехода $T$ среды.

$$T(s_t, a_t) \mapsto s_{t+1}$$

In [4]:
states = set(range(3))
actions = set(range(1))

print(f'{states=} | {actions=}')


T = {  # (s, a)
    (0, 0): 1,
    (1, 0): 2,
    (2, 0): 2
}
print(f'Transition function {T=}')

A_mdp = states, actions, T

states={0, 1, 2} | actions={0}
Transition function T={(0, 0): 1, (1, 0): 2, (2, 0): 2}


Попробуйте записать функцию перехода в матричном виде:

In [5]:
def get_transition_prob(T, a, s1, s2):
    num = sum(1 for k, v in T.items() if k == (s1, a) and v == s2)
    den = sum(1 for k, v in T.items() if k == (s1, a))
    return num / den if den > 0 else 0

def get_transition_matrix(A_mdp):
    states, actions, T = A_mdp
    n, m = len(actions), len(states)
    
    P = np.zeros((n, m, m))
    
    for a in range(n):
        for s1 in states:
            for s2 in states:
                P[a, s1, s2] = get_transition_prob(T, a, s1, s2)
    return P
    
P = get_transition_matrix(A_mdp)
P

array([[[0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.]]])

Как получить вероятность нахождения агента в состоянии (1) через N шагов? Что происходит с вероятностями нахождения в состояниях при $N \rightarrow \infty$

Вероятности считаются рекурсивно:

$$\mathbb{P}(s_N = s') = \sum_{s \in S}\sum_{a \in A}\pi(a|s)\mathbb{P}(s_N = s' | s_{N - 1} = s, a_{N - 1} = a)\mathbb{P}(s_{N-1} = s)$$

* первый множитель: $\pi$, не зависит от времени
* второй множитель: чиселко $P_{a,s,s'}$ из матрицы переходов
* третий множитель: вероятности на предыдущем шаге оказаться в $s$, рекурсия в этом месте

Будем считать, что агент ходит равновероятно по любому ребру. Для подсчетов будет достаточно. Реализация неэффективная, так как можно было бы через ДП посчитать или кэшировать функцию.

In [6]:
def get_prob_of_state(A_mdp, P, first_state, k, N):  # вероятность быть в состоянии (k) после N шагов
    if N == first_state:
        return float(k == first_state)

    prob = 0
    states, actions, T = A_mdp
    for s in states:
        n_actions = sum(1 for key in T if key[0] == s)  # число возможных действий из состояния s
        if n_actions == 0:
            continue

        cur_prob = 0
        for a in actions:
            cur_prob += P[a, s, k] * get_prob_of_state(A_mdp, P, first_state, s, N - 1)  # вероятность быть в s на шаге N - 1
        prob += 1 / n_actions * cur_prob
    return prob

In [7]:
k = 1
for N in range(5):
    print(f"Вероятность оказаться в состоянии ({k}) через {N} шагов: {get_prob_of_state(A_mdp, P, 0, k, N):.2f}")

Вероятность оказаться в состоянии (1) через 0 шагов: 0.00
Вероятность оказаться в состоянии (1) через 1 шагов: 1.00
Вероятность оказаться в состоянии (1) через 2 шагов: 0.00
Вероятность оказаться в состоянии (1) через 3 шагов: 0.00
Вероятность оказаться в состоянии (1) через 4 шагов: 0.00


Результат оправданный - мы гарантированно будем в $(1)$ после первого шага, а затем выйдем и не вернемся в него.

- B: <img src=https://i.ibb.co/GQ2tVtjC/mdp-b.png caption="B" width="400" height="100">

Аналогичное для $B$:

In [8]:
states = set(range(4))
actions = set(range(3))

print(f'{states=} | {actions=}')


T = {  # (s, a)
    (0, 0): 1,
    (0, 1): 2,
    (0, 2): 3,
    
    (1, 0): 1,
    (1, 1): 1,
    (1, 2): 1,

    (2, 0): 2,
    (2, 1): 2,
    (2, 2): 2,

    (3, 0): 3,
    (3, 1): 3,
    (3, 2): 3,
}
print(f'Transition function {T=}')

A_mdp = states, actions, T

states={0, 1, 2, 3} | actions={0, 1, 2}
Transition function T={(0, 0): 1, (0, 1): 2, (0, 2): 3, (1, 0): 1, (1, 1): 1, (1, 2): 1, (2, 0): 2, (2, 1): 2, (2, 2): 2, (3, 0): 3, (3, 1): 3, (3, 2): 3}


In [9]:
P = get_transition_matrix(A_mdp)
P

array([[[0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]])

In [10]:
for N in range(2):
    for k in range(4):
        print(f"Вероятность оказаться в состоянии ({k}) через {N} шагов: {get_prob_of_state(A_mdp, P, 0, k, N):.2f}")

Вероятность оказаться в состоянии (0) через 0 шагов: 1.00
Вероятность оказаться в состоянии (1) через 0 шагов: 0.00
Вероятность оказаться в состоянии (2) через 0 шагов: 0.00
Вероятность оказаться в состоянии (3) через 0 шагов: 0.00
Вероятность оказаться в состоянии (0) через 1 шагов: 0.00
Вероятность оказаться в состоянии (1) через 1 шагов: 0.33
Вероятность оказаться в состоянии (2) через 1 шагов: 0.33
Вероятность оказаться в состоянии (3) через 1 шагов: 0.33


Задайте еще несколько MDP:

- C: <img src=https://i.ibb.co/Jj9LYHjP/mdp-c.png caption="C" width="400" height="100">

In [11]:
states = set(range(4))
actions = set(range(2))

print(f'{states=} | {actions=}')


T = {  # (s, a)
    (0, 0): 1,
    (0, 1): 2,

    (1, 0): 1,
    (1, 1): 3,

    (2, 0): 3,
    (2, 1): 2,

    (3, 0): 3,
    (3, 1): 3
}
print(f'Transition function {T=}')

A_mdp = states, actions, T

states={0, 1, 2, 3} | actions={0, 1}
Transition function T={(0, 0): 1, (0, 1): 2, (1, 0): 1, (1, 1): 3, (2, 0): 3, (2, 1): 2, (3, 0): 3, (3, 1): 3}


In [12]:
P = get_transition_matrix(A_mdp)
P

array([[[0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.]],

       [[0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]])

In [13]:
k = 3
for N in range(8):
        print(f"Вероятность оказаться в состоянии ({k}) через {N} шагов: {get_prob_of_state(A_mdp, P, 0, k, N):.2f}")

Вероятность оказаться в состоянии (3) через 0 шагов: 0.00
Вероятность оказаться в состоянии (3) через 1 шагов: 0.00
Вероятность оказаться в состоянии (3) через 2 шагов: 0.50
Вероятность оказаться в состоянии (3) через 3 шагов: 0.75
Вероятность оказаться в состоянии (3) через 4 шагов: 0.88
Вероятность оказаться в состоянии (3) через 5 шагов: 0.94
Вероятность оказаться в состоянии (3) через 6 шагов: 0.97
Вероятность оказаться в состоянии (3) через 7 шагов: 0.98


Давайте попробуем задать двух агентов: случайного и оптимального (для каждой среды свой).

In [14]:
class Agent:
    def __init__(self, actions):
        self.rng = np.random.default_rng()
        self.actions = np.array(list(actions))
        
        self.policy = None
    
    def set_policy(self, policy):
        self.policy = policy

    def act(self, state, strategy="random"):
        if strategy == "random":
            return int(self.rng.integers(len(self.actions)))
        return self.policy[state]

В качестве дополнения, запишите стратегию агента

Сделаю для $C$, в предположении, что хотим как можно быстрее добраться до терминальной точки.

In [15]:
optimal_policy_C = {
    0: 0,
    1: 1,
    2: 0
}

In [16]:
agent = Agent(actions)
agent.set_policy(optimal_policy_C)
print(agent.act(1, "random"))
print(agent.act(1, "random"))
print(agent.act(1, "random"))
print(agent.act(1, "random"))
print(agent.act(1, "random"))
print(agent.act(1, "random"))
print(agent.act(1, "optimal"))

1
0
0
0
0
1
1
1
0
0
0
0
1
1


## Reward, reward function

Теперь добавим произвольную функцию вознаграждения. Например, для A:

 *Я для C переделаю*

- C: <img src=https://i.ibb.co/Jj9LYHjP/mdp-c.png caption="C" width="400" height="100">

In [17]:
R = {  # (s, a)
    (0, 0): -10.0,
    (0, 1): -1.0,

    (1, 0): -1.0,
    (1, 1): +100.0,

    (2, 0): +10.0,
    (2, 1): -1.0,

    (3, 0): 0,
    (3, 1): 0
}
print(R)

A_mdp = *A_mdp, R
print(A_mdp)

{(0, 0): -10.0, (0, 1): -1.0, (1, 0): -1.0, (1, 1): 100.0, (2, 0): 10.0, (2, 1): -1.0, (3, 0): 0, (3, 1): 0}
({0, 1, 2, 3}, {0, 1}, {(0, 0): 1, (0, 1): 2, (1, 0): 1, (1, 1): 3, (2, 0): 3, (2, 1): 2, (3, 0): 3, (3, 1): 3}, {(0, 0): -10.0, (0, 1): -1.0, (1, 0): -1.0, (1, 1): 100.0, (2, 0): 10.0, (2, 1): -1.0, (3, 0): 0, (3, 1): 0})


## Interaction loop, trajectory, termination, truncation, episode

Общий цикл взаимодействия в рамках эпизода:
1. Инициализировать среду: $s \leftarrow \text{env.init()}$
2. Цикл:
    - выбрать действие: $a \leftarrow \pi(s)$
    - получить ответ от среды: $s, r, d \leftarrow \text{env.next(a)}$
    - если $d == \text{True}$, выйти из цикла

In [18]:
def run_episode(mdp):
    states, actions, T, R = mdp
    agent = Agent(actions)

    s = 0
    tau = []
    for _ in range(5):
        a = agent.act(s)
        s_next = T[(s, a)]
        r = R[(s, a)]

        tau.append((s, a, r))
        s = s_next

    return tau

run_episode(A_mdp)

[(0, 1, -1.0), (2, 0, 10.0), (3, 1, 0), (3, 0, 0), (3, 0, 0)]

Termination — означает окончание эпизода, когда достигнуто терминальное состояние. Является частью задания среды.

Truncation — означает окончание эпизода, когда достигнут лимит по числу шагов (=времени). Обычно является внешне заданным параметром для удобства обучения.

Пока не будем вводить truncation, но поддержим termination: расширьте определение среды информацией о терминальных состояниях для всех описанных ранее сред. Сгенерируйте по несколько случайных траекторий для каждой среды.

In [19]:
terminal_states = {3,}
states, actions, T, R = A_mdp
A_mdp = states, terminal_states, actions, T, R

In [20]:
def run_episode_with_policy(mdp, strategy="random", policy=None, truncation=10):
    states, terminal_states, actions, T, R = mdp
    agent = Agent(actions)
    if strategy != "random":
        agent.set_policy(policy)

    s = 0
    tau = []
    for _ in range(truncation):
        a = agent.act(s, strategy)
        s_next = T[(s, a)]
        r = R[(s, a)]

        tau.append((s, a, r))
        s = s_next
        
        if s in terminal_states:
            break

    return tau

Сгенерируем рандомные траектории:

In [21]:
random_trajectories = []
for _ in range(10):
    tau = run_episode_with_policy(A_mdp, strategy="random", truncation=10)
    random_trajectories.append(tau)
    print(tau)

[(0, 1, -1.0), (2, 1, -1.0), (2, 0, 10.0)]
[(0, 0, -10.0), (1, 1, 100.0)]
[(0, 0, -10.0), (1, 1, 100.0)]
[(0, 1, -1.0), (2, 1, -1.0), (2, 1, -1.0), (2, 1, -1.0), (2, 0, 10.0)]
[(0, 1, -1.0), (2, 1, -1.0), (2, 1, -1.0), (2, 0, 10.0)]
[(0, 0, -10.0), (1, 0, -1.0), (1, 0, -1.0), (1, 0, -1.0), (1, 0, -1.0), (1, 1, 100.0)]
[(0, 0, -10.0), (1, 1, 100.0)]
[(0, 0, -10.0), (1, 1, 100.0)]
[(0, 1, -1.0), (2, 1, -1.0), (2, 1, -1.0), (2, 1, -1.0), (2, 0, 10.0)]
[(0, 0, -10.0), (1, 0, -1.0), (1, 1, 100.0)]


Оптимальная траектория:

In [22]:
optimal_trajectory = run_episode_with_policy(A_mdp, strategy="optimal", policy=optimal_policy_C, truncation=10)
optimal_trajectory

[(0, 0, -10.0), (1, 1, 100.0)]

### Return, expected return

Наиболее важная метрика оценки качества работы агента: отдача.

Отдача: $G(s_t) = \sum_{i=t}^T r_i$

Обычно также вводят параметр $\gamma \in [0, 1]$, дисконтирующий будущие вознаграждения. А еще, тк отдача может меняться от запуска к запуску благодаря вероятностным процессам, нас интересует отдача в среднем — ожидаемая отдача:

$$\hat{G}(s_t) = \mathbb{E} [ \sum_{i=t}^T \gamma^{i-t} r_i ]$$

Именно ее и оптимизируют в RL.

Давайте научимся считать отдачу для состояний по траектории и считать среднюю отдачу.

In [23]:
def get_returns(states, trajectories, gamma):
    zeros = [0 for _ in range(len(states))]
    returns = dict(zip(states, zeros))
    counts  = dict(zip(states, zeros))
    for tau in trajectories:
        for i, (s, a, r) in enumerate(tau):
            returns[s] += r * gamma ** i
            counts[s]  += 1

    def get_return(s):
        return returns[s] / max(1, counts[s])
    
    return dict(zip(states, list(map(get_return, states))))

In [24]:
gamma = 0.8

In [25]:
get_returns(states, random_trajectories, gamma)

{0: -6.4, 1: 37.600581818181816, 2: 1.0436923076923081, 3: 0.0}